In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import torch
import numpy as np
import manify
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression

In [3]:
# Basic GCN implementation


class VanillaGCNLayer(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.W = torch.nn.Parameter(torch.randn(in_features, out_features) * 0.01)

    def forward(self, X, A_hat):
        return A_hat @ X @ self.W


class VanillaGCN(torch.nn.Module):
    def __init__(self, dim, classes, num_hidden=4):
        super().__init__()
        self.layers = torch.nn.ModuleList()
        for i in range(num_hidden):
            self.layers.append(VanillaGCNLayer(dim, dim))
            self.layers.append(torch.nn.ReLU())
        self.layers.append(torch.nn.Linear(dim, classes))

    def forward(self, X, A_hat):
        for layer in self.layers:
            if isinstance(layer, VanillaGCNLayer):
                X = layer(X, A_hat)
            else:
                X = layer(X)
        return X

In [ ]:
SEEDS = 10
LR = 0.01
EPOCHS = 1_000
BIAS = False
RBF_SCALE = 10  # Divide by this to get a reasonable scale for the RBF kernel

results = []
pm = manify.ProductManifold(signature=[(0, 2)], stereographic=True)


def train_model(model, X_train, y_train, A_hat=None, num_epochs=EPOCHS, lr=LR):
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    progress_bar = tqdm(total=num_epochs)

    for i in range(num_epochs):
        # Forward pass
        if A_hat is not None:
            y_pred = model(X_train, A_hat)
        else:
            y_pred = model(X_train)

        # Compute loss
        loss = loss_fn(y_pred, y_train)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update progress bar
        progress_bar.update(1)
        progress_bar.set_postfix(loss=loss.item())

    return model


def evaluate_model(model, X_test, y_test, A_hat=None):
    with torch.no_grad():
        if A_hat is not None:
            y_pred = model(X_test, A_hat)
        else:
            y_pred = model(X_test)

        acc = (y_pred.argmax(dim=1) == y_test).float().mean()
    return acc.item()


for seed in range(SEEDS):
    print(f"Seed: {seed}")
    # Get models
    torch.manual_seed(seed)
    kappa_gcn = manify.KappaGCN(pm=pm, num_hidden=4, output_dim=3)

    torch.manual_seed(seed)
    kappa_mlp = manify.KappaGCN(pm=pm, num_hidden=4, output_dim=3)

    torch.manual_seed(seed)
    kappa_mlr = manify.KappaGCN(pm=pm, num_hidden=0, output_dim=3)

    torch.manual_seed(seed)
    torch_gcn = VanillaGCN(dim=2, classes=3, num_hidden=4)

    torch.manual_seed(seed)
    torch_mlp = torch.nn.Sequential(
        torch.nn.Linear(2, 2, bias=BIAS),
        torch.nn.ReLU(),
        torch.nn.Linear(2, 2, bias=BIAS),
        torch.nn.ReLU(),
        torch.nn.Linear(2, 2, bias=BIAS),
        torch.nn.ReLU(),
        torch.nn.Linear(2, 2, bias=BIAS),
        torch.nn.ReLU(),
        torch.nn.Linear(2, 3),
    )
    for layer in torch_mlp:
        if isinstance(layer, torch.nn.Linear):
            layer.weight = torch.nn.Parameter(torch.randn(layer.weight.shape) * 0.01)

    torch.manual_seed(seed)
    torch_mlr = torch.nn.Linear(2, 3)

    sklearn_mlr = LogisticRegression(
        solver="lbfgs", max_iter=EPOCHS, fit_intercept=True, penalty=None,
    )

    # Generate data
    X, y = pm.gaussian_mixture(num_points=1000, num_classes=3, seed=seed)

    # Get adjacency matrix
    A = manify.predictors.kappa_gcn.get_A_hat(torch.exp(-pm.pdist2(X) / RBF_SCALE))

    # Train models...
    # GCNs
    # NB: divide by 4 because KappaGCN logits are scaled by 4 by default
    kappa_gcn.fit(X, y, A=A, lr=LR / 4, epochs=EPOCHS)
    kappa_gcn_acc = evaluate_model(kappa_gcn, X, y, A_hat=A)

    torch_gcn = train_model(torch_gcn, X, y, A_hat=A)
    torch_gcn_acc = evaluate_model(torch_gcn, X, y, A_hat=A)

    # MLPs
    kappa_mlp.fit(X, y, A=None, lr=LR / 4, epochs=EPOCHS)
    kappa_mlp_acc = evaluate_model(kappa_mlp, X, y, A_hat=None)

    torch_mlp = train_model(torch_mlp, X, y)
    torch_mlp_acc = evaluate_model(torch_mlp, X, y)

    # MLRs
    kappa_mlr.fit(X, y, A=None, lr=LR / 4, epochs=EPOCHS)
    kappa_mlr_acc = evaluate_model(kappa_mlr, X, y, A_hat=None)

    torch_mlr = train_model(torch_mlr, X, y)
    torch_mlr_acc = evaluate_model(torch_mlr, X, y)

    sklearn_mlr.fit(X.numpy(), y.numpy())
    sklearn_acc = sklearn_mlr.score(X.numpy(), y.numpy())

    # Store results
    results += [
        {
            "seed": seed,
            "kappa_gcn_acc": kappa_gcn_acc,
            "torch_gcn_acc": torch_gcn_acc,
            "kappa_mlp_acc": kappa_mlp_acc,
            "torch_mlp_acc": torch_mlp_acc,
            "kappa_mlr_acc": kappa_mlr_acc,
            "torch_mlr_acc": torch_mlr_acc,
            "sklearn_mlr_acc": sklearn_acc,
        }
    ]

results_df = pd.DataFrame(results)
results_df

Seed: 0


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

,seed,kappa_gcn_acc,torch_gcn_acc,kappa_mlp_acc,torch_mlp_acc,kappa_mlr_acc,torch_mlr_acc,sklearn_mlr_acc
0,0,0.906,0.591,0.918,0.591,0.903,0.904,0.903
1,1,0.433,0.433,0.433,0.531,0.519,0.519,0.519
2,2,0.204,0.517,0.204,0.517,0.595,0.595,0.595
3,3,0.651,0.651,0.650,0.651,0.669,0.669,0.669
4,4,0.633,0.456,0.680,0.677,0.780,0.780,0.780
5,5,0.573,0.625,0.669,0.669,0.626,0.626,0.626
6,6,0.390,0.793,0.390,0.804,0.781,0.781,0.781
7,7,0.479,0.479,0.638,0.479,0.675,0.675,0.675
8,8,0.654,0.654,0.654,0.654,0.803,0.802,0.802
9,9,0.665,0.602,0.700,0.602,0.655,0.655,0.655


In [14]:
from scipy.stats import wilcoxon
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

print("\t" + "\t".join(results_df.columns[1:]))
for col in results_df.columns:
    if col == "seed":
        continue
    print(col, end="\t")
    for col2 in results_df.columns:
        if col2 == "seed":
            continue
        print(f"{wilcoxon(results_df[col], results_df[col2]).pvalue:.4f}", end="\t")
    print()

	kappa_gcn_acc	torch_gcn_acc	kappa_mlp_acc	torch_mlp_acc	kappa_mlr_acc	torch_mlr_acc	sklearn_mlr_acc
kappa_gcn_acc	1.0000	1.0000	0.0625	0.3750	0.0098	0.0098	0.0098	
torch_gcn_acc	1.0000	1.0000	0.7422	0.1250	0.0059	0.0059	0.0059	
kappa_mlp_acc	0.0625	0.7422	1.0000	0.9453	0.0820	0.0820	0.0840	
torch_mlp_acc	0.3750	0.1250	0.9453	1.0000	0.0488	0.0488	0.0488	
kappa_mlr_acc	0.0098	0.0059	0.0820	0.0488	1.0000	1.0000	0.5547	
torch_mlr_acc	0.0098	0.0059	0.0820	0.0488	1.0000	1.0000	0.6250	
sklearn_mlr_acc	0.0098	0.0059	0.0840	0.0488	0.5547	0.6250	1.0000	


In [6]:
from scipy.stats import pearsonr

print("\t" + "\t".join(results_df.columns[1:]))
for col in results_df.columns:
    if col == "seed":
        continue
    print(col, end="\t")
    for col2 in results_df.columns:
        if col2 == "seed":
            continue
        print(f"{pearsonr(results_df[col], results_df[col2]).statistic:.4f}", end="\t")
    print()

	kappa_gcn_acc	torch_gcn_acc	kappa_mlp_acc	torch_mlp_acc	kappa_mlr_acc	torch_mlr_acc	sklearn_mlr_acc
kappa_gcn_acc	1.0000	0.0493	0.9992	0.0660	0.5668	0.5676	0.5670	
torch_gcn_acc	0.0493	1.0000	0.0454	0.7737	0.3605	0.3597	0.3603	
kappa_mlp_acc	0.9992	0.0454	1.0000	0.0693	0.5814	0.5822	0.5816	
torch_mlp_acc	0.0660	0.7737	0.0693	1.0000	0.4480	0.4469	0.4480	
kappa_mlr_acc	0.5668	0.3605	0.5814	0.4480	1.0000	1.0000	1.0000	
torch_mlr_acc	0.5676	0.3597	0.5822	0.4469	1.0000	1.0000	1.0000	
sklearn_mlr_acc	0.5670	0.3603	0.5816	0.4480	1.0000	1.0000	1.0000	


In [7]:
# 	kappa_mlp_acc	torch_mlp_acc	kappa_mlr_acc	torch_mlr_acc	sklearn_mlr_acc
# kappa_mlp_acc	1.0000	0.9375	0.0820	0.0840	0.0840
# torch_mlp_acc	0.9375	1.0000	0.0098	0.0098	0.0098
# kappa_mlr_acc	0.0820	0.0098	1.0000	1.0000	0.4902
# torch_mlr_acc	0.0840	0.0098	1.0000	1.0000	0.5391
# sklearn_mlr_acc	0.0840	0.0098	0.4902	0.5391	1.0000